# Exploring Mental Health Discourse on Reddit

### Import Libraries 

In [1]:
import os
import datetime
import time
import re
import requests
import random

import pandas as pd
import praw

from nltk.corpus import stopwords

import string 
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist

/Users/samantharivas/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/samantharivas/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [35]:
import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/samantharivas/nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/samantharivas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samantharivas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/samantharivas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Connecting to Reddit API

*The code was modified based on resources available at https://github.com/rogerfitz/tutorials/blob/master/subreddit_analysis/0_Setup.ipynb and a tutorial accessed via https://www.youtube.com/watch?v=a3zQdlLwEb4.*

In [3]:
# establish connection with reddit API
reddit = praw.Reddit(
    client_id="81TBuwGeFdL5ZfijNpLxNw",
    client_secret="fn2ztcn8ojr3vewO93h46nZ2YY47mw",
    user_agent="Academic_Winner_5303",
)

#### Obtaining Posts from r/mentalhealth

In [4]:
# specify the subreddit
subreddit_mh = 'MentalHealth'

# fetch posts fromsubreddit
subreddit = reddit.subreddit(subreddit_mh)
word_to_check = "mental health"

In [5]:
for submission in reddit.subreddit('mentalhealth').hot(limit=None):
    print(submission.title)

Men's Mental Health Month
Warning regarding DM's and chatgroups offering "help".
You do not have to tell people “why” you are in therapy.
how do you get over losing a job over your poor mental health 
Everyday is a struggle fighting depression... but I'm trying 🥲
I think I’m going to start screaming in my car
What's your toxic trait? 
How to get over the feeling of thinking like you’re no allowed to exist or be in public spaces?
Am I just supposed to stay single?
Women around my age make me uncomfortable 
Do you feel like having a therapist and a psychiatrist is  like paying for a support system?
my anxiety is ruining me
Is any of this ‘normal’ or should i see a doctor?
Therapist. But why? 
Test day panic attack
How do I heal from my childhood trauma?
My anxiety drives me insane
How do you get over trust issues and overthinking? 
I think I just completely snapped
I need to talk
Anyone that feels the same in early 20s?
I am driving myself crazy with these thoughts of becoming rich.
How 

I kinda suck
Hey can you tell me why everyone hates me. 
How to really not care what other people think
i don’t understand what I’m feeling.
Is it possible to talk to my doctor about getting help for my mental health without my parents knowing?
Just started meds 
Being overly emotional and just exhausted 
what movies/tv shows/podcasts/music/anything do you watch/listen to when sad, happy, anxious, ANYTHING
Existence Dilemmas
Help with insurance and seeking treatment.
I hated growing up in my rich family
Urges to kill myself and not understanding anything
How to prepare loved ones
is this derealisation?
there is nothing to hope for I am exhausted 
I need other’s opinions
Advice before I let my eating disorder ruin my life for good. 
Men of Reddit, how do you release your emotions? Do you find crying to help?
I have no sense of self, is this normal for anxiety/depression?
Why me, why me?
My mental health has been getting way worse
I just need answers
I keep stalking people who i wronged 

Life with ADHD 
I don't feel good and I think I have a real problem. 
I am self destructing and I don't know what I can do about it. Therapy has made everything worse, I need help.
i feel physically uncomfortable at all times
Watch your triggers each morning. When an attack happens, make some green tea. Take some deep breaths. Eat the rainbow, our eyes are designed to spot fruit in the distance. Fruit, veg, whole grains, nuts, seeds, berries, legumes, herbs and spices. Learn the proper push up technique. 
i need help please this is getting really fucking bad
Need support/help - depressive episode 
Am I dealing with PTSD?
Number for mental health support and signposting - northwest of England 
 Why do I break down and cry at every little inconvenience? 
I think I'm too sensitive
why does it feel like the devil is in my head
Any advice dealing with health condition based frustration?
The thing about trauma
What is a good excuse as to my lack of work history? (Its because I couldn't handl

I don’t even know what medication to try next…
I just really miss her 
Returned to my old depression habits
Mental Health is bad right nka
I need help with my Imposter Syndrome 
I feel like Im going insane
why is mental health suddenly unimportant when the person suffering is aware of their mental illness
I'm a doormat - I don't like myself
How do you fight a victim mentality?
Is this stress or mental illness?
Bad experience with Effexor - is Wellbutrin a safe alternative for me?
Inpatient care in Quebec or Ontario
Mindfulness OP
Sick of my job and i feel powerless.
beating depression?
How to be productive with debilitating anxiety?
How to let go your toxic friends ?
School advice and help
What is it called (this has been happening a lot lately)...
Am I just way too sensitive? Do I overthink?
How to stop the pain
My sister hates my (depression?) room and criticizes it every visit
I can’t tell if my girlfriend wants to be with me. What should I do??
My mental health is at its worst.. 
W

I am unhealthily attached to a teacher that left a year ago, what's wrong with me?
Am i the only who can’t hate anyone at all?
I've cried almost every night for a month because of loneliness, this is new for me, and I don't know how to stop it
Why do i feel like this
-New neighbour downstairs already aggressively banging on the ceiling complaining about noise twice in the space of 24 hours. What should I do?
Fed up of life
Just got out of a toxic relationship and I feel so disconnected from my friends
Are intrusive thoughts always about hurting yourself and/or others?
I can't figure out what I'm feeling/struggling with 
Is it normal to feel like this? It shouldn’t be
My brother is the reason I want to end my life
How to enjoy your own success instead of searching new challenges immidiately?
When it rains it pours 
idk if I have depression........
My boyfriend was baker acted 
Brushed off?
Idk what to think. Nothing would be best. Need opinion. 
What is wrong with me?
I dont know whats 

In [6]:
subreddit=reddit.subreddit("mentalhealth")
subreddit.over18

False

In [7]:
subreddit.subscribers

468335

In [8]:
subreddit.__dict__

{'display_name': 'mentalhealth',
 '_reddit': <praw.reddit.Reddit at 0x7f9e54739af0>,
 '_fetched': True,
 '_path': 'r/mentalhealth/',
 'user_flair_background_color': None,
 'submit_text_html': '<!-- SC_OFF --><div class="md"><p>All promotional, survey, research study, or business posts require moderator approval. Please message a moderator to check if your post is suitable. Avoid posting direct links without some context.</p>\n</div><!-- SC_ON -->',
 'restrict_posting': True,
 'user_is_banned': None,
 'free_form_reports': True,
 'wiki_enabled': True,
 'user_is_muted': None,
 'user_can_flair_in_sr': None,
 'header_img': None,
 'title': 'Mental Health',
 'allow_galleries': False,
 'icon_size': None,
 'primary_color': '#005ba1',
 'active_user_count': 53,
 'icon_img': '',
 'display_name_prefixed': 'r/mentalhealth',
 'accounts_active': 53,
 'public_traffic': False,
 'subscribers': 468335,
 'user_flair_richtext': [],
 'name': 't5_2qirg',
 'quarantine': False,
 'hide_ads': False,
 'prediction_

In [9]:
# define function to fetch posts from subreddit ('MentalHealthSupport')
def fetch_posts(subreddit, category='hot', limit=None):
    if category == 'top':
        posts = subreddit.top(time_filter='year', limit=limit)
    elif category == 'rising':
        posts = subreddit.rising(limit=limit)
    elif category == 'new':
        posts = subreddit.new(limit=limit)
    else:
        posts = subreddit.hot(limit=limit)
    
    rows = []
    for submission in posts:
        row = {
            'title': submission.title,
            'score': submission.score,
            'upvote_ratio': submission.upvote_ratio,
            'created_utc': submission.created_utc,
            'selftext': submission.selftext,
            'subreddit': submission.subreddit,
            'author': str(submission.author),
            'media_only': submission.media_only,
            'permalink': submission.permalink,
            'num_comments': submission.num_comments
        }
        rows.append(row)
    
    return pd.DataFrame(rows)

In [10]:
mental_health_top = fetch_posts(subreddit, category='top')
mental_health_hot = fetch_posts(subreddit, category='hot')
mental_health_rising = fetch_posts(subreddit, category='rising')
mental_health_new = fetch_posts(subreddit, category='new')

In [11]:
# concatenate dataframes
mental_health_df = pd.concat([mental_health_top,
                               mental_health_hot, 
                               mental_health_rising, 
                               mental_health_new],
                              ignore_index=True)

# drop duplicates
mental_health_df = mental_health_df.drop_duplicates()

In [12]:
mental_health_df['upvotes']=mental_health_df['score']/mental_health_df['upvote_ratio']
mental_health_df['downvotes']=mental_health_df['upvotes']-mental_health_df['score']
mental_health_df['upvotes/subscribers']=mental_health_df['upvotes']/subreddit.subscribers

mental_health_df.sample(5)

,title,score,upvote_ratio,created_utc,selftext,subreddit,author,media_only,permalink,num_comments,upvotes,downvotes,upvotes/subscribers
865,How to bring your brain back to health after d...,39,0.98,1.694772e+09,Have lost much of my brain power after depress...,mentalhealth,nothingarc,False,/r/mentalhealth/comments/16j94dr/how_to_bring_...,21,39.795918,0.795918,0.000085
955,my anxiety is ruining me,3,1.00,1.718233e+09,i’ve had anxiety my WHOLE life. even things fr...,mentalhealth,rosie200416,False,/r/mentalhealth/comments/1dek7ij/my_anxiety_is...,0,3.000000,0.000000,0.000006
1861,Emotionally repressed,1,1.00,1.717917e+09,"Idk if I’m venting or it’s like a Dairy entry,...",mentalhealth,Piplup_supremacy,False,/r/mentalhealth/comments/1dbnsj0/emotionally_r...,1,1.000000,0.000000,0.000002
1309,Should I tell my parents about sh?,1,1.00,1.718124e+09,My parents aren’t best with mental health. The...,mentalhealth,vxviolet,False,/r/mentalhealth/comments/1ddittj/should_i_tell...,4,1.000000,0.000000,0.000002
806,What YouTubers make you feel good?,43,0.98,1.704768e+09,"Hi all, I am looking to find some new YouTuber...",mentalhealth,teenytinypeanut,False,/r/mentalhealth/comments/1923rip/what_youtuber...,119,43.877551,0.877551,0.000094


In [13]:
mental_health_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2033 entries, 0 to 2894
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                2033 non-null   object 
 1   score                2033 non-null   int64  
 2   upvote_ratio         2033 non-null   float64
 3   created_utc          2033 non-null   float64
 4   selftext             2033 non-null   object 
 5   subreddit            2033 non-null   object 
 6   author               2033 non-null   object 
 7   media_only           2033 non-null   bool   
 8   permalink            2033 non-null   object 
 9   num_comments         2033 non-null   int64  
 10  upvotes              2033 non-null   float64
 11  downvotes            2033 non-null   float64
 12  upvotes/subscribers  2033 non-null   float64
dtypes: bool(1), float64(5), int64(2), object(5)
memory usage: 208.5+ KB


In [16]:
# create directory to store data 
os.makedirs('data', exist_ok=True)

# create csv 
mental_health_df.to_csv('data/mental_health_posts.csv', index=False)

#### Obtaining Posts  from r/MentalHealthSupport 

In [17]:
# specify the subreddit
subreddit_mhs = 'MentalHealthSupport'

# fetch posts fromsubreddit
subreddit = reddit.subreddit(subreddit_mhs)
word_to_check = "mental health"

In [18]:
for submission in reddit.subreddit('mentalhealthsupport').hot(limit=None):
    print(submission.title)

How do I even stop pushing people away?
How to deal with grief and shame as the other women?
I feel mentally retarded
ECT related memory loss
Everything is falling apart 
Does it ever get better?(Male: 25)
How do I overcome my SA and what I did to my partner because of my trauma?
0 confidence, 0 motive, barely alive.
TLDR: How do I improve myself if I keep losing the whole reason to do anything. My friends.
I’m loosing my comfort person tomorrow
Health Insomnia?
Mytherapist told me expecting reciprocity in a friendship is selfish
Hiw do you deal with depression?
How do you guys cope with the cruel absurdity of life?
What would you do?
Depressed
Need help with anxiety/ocd
Keep rude comments to yourself 
How to handle suicide loss?
Bipolar Housing help
Mental health support for family member
my parents have been the cause of my unhappiness
Has anybody processed through some of their condition or trauma & realised life is very unaffected?
What’s the point 
Idk, I just need support, and so

Very scared of brain eating ameoba after tap water from work went into my nose.
Breaking Free from Conditioning: Seeing Beyond the Frame
God I hate my social anxiety disorder!
mental health help
The push and pull
Going back to school as a "mature student" and feeling weird about it
Mental Health Journey 
Mental Health & Divorce
Anxiety
I pushed my friends away because I fear I myself will destroy them. Now I feel like letting it all go.
I think today I realize I totally hate myself
Shocked my therapist
Question: is it just me or has life been really lifing lately?
Do I have bipolar?
I need an advice on how to save my life

do you ever not want to feel better?
Random heartache + apathy
I feel worn out 
Extreme worry about police academy 
how do i come out of my depression
My situation (I need help)
Long time sufferer of anxiety just lost my cat, I don’t know what to do
Feeling Anxiety
Im struggling rn
Thinking of hospitalizing myself.
Expressing my feelings about my parents
feeling guil

19M, beginning to really struggle and i’m worried.
Dad faked suicidal ideations as a ploy for attention when I was 15, and I feel emotionally stunted in large part because of it
It's 1am and worrying about my dad's health 
Stable then unstable 
Bitter and alone
Mental Health 
Asking for similar experiences
So I’m new here but just looking for advice I guess. 
I feel crazy
Skin picking TW; SH
I feel like everyone is using me
nothing feels real (rant/whats going on?)
I'm struggling so much...
I feel like I’m ruining everyone’s lives
Having a hard time processing my moms illness
Intrusive Thoughts
Is this really as good as mental health gets
Coping mechanisms are not as advertised??! Nobody told me!
Over-Pampered
I hate myself
Emotions
Help I can’t stop stealing
Is it worth pursuing fame? 21 M- Story Time/ Question
How to ask for help
A colleague who has caused a lot of stress in my life and I am leaving my job because of them is telling me that they're suicidal. I don't know how to help 

Worried 
Violent thoughts help please read all if you can
How to get over/detach from someone extremely close?
I cant seem to listen
Why is my dad so weirdly mean to me?
I am doing my dream job, but I started having these anxiety attacks and don’t want to go to work
Why do I feel embarrassed over being open with other people?
22F Need to Vent Not feeling great. 
I am tired. 
 Feeling Drowned and Unmotivated - Looking for Support
What happened to my life
I'm... exhausted - Failed attempt at poetry 
What is this? I am not sure.
I'm losing hope.  Have others found a way out?
I need help, as a Canadian where is the best place for therapy?
I (22m) am struggling and don’t want to talk to my GF(23F) about any of it. 
[URGENT] How do I deal with a suicidal friend?
What do you think?
I feel like such a failure 
PTSD relapse or Depression spiral??
Does anyone else chronically talk to themselves? 
What turned things around for you?
what is wrong with me?
I’m so tired all the time and I think it’s

In [19]:
subreddit=reddit.subreddit('mentalhealthsupport')
subreddit.over18

False

In [20]:
subreddit.subscribers

46119

In [21]:
subreddit.__dict__

{'display_name': 'MentalHealthSupport',
 '_reddit': <praw.reddit.Reddit at 0x7f9e54739af0>,
 '_fetched': True,
 '_path': 'r/MentalHealthSupport/',
 'user_flair_background_color': None,
 'submit_text_html': '<!-- SC_OFF --><div class="md"><p>Please provide context to links since they require moderator approval. Promotional posts are not permitted.  Message the moderators if you have any query. Make sure your post follows the rules.</p>\n\n<p>This subreddit is optimized for the new Reddit redesign and official app. Please use the newer design to enjoy the subreddit to its fullest, thank you!</p>\n</div><!-- SC_ON -->',
 'restrict_posting': True,
 'user_is_banned': None,
 'free_form_reports': True,
 'wiki_enabled': True,
 'user_is_muted': None,
 'user_can_flair_in_sr': None,
 'header_img': None,
 'title': 'Mental Health Support & Azal',
 'allow_galleries': False,
 'icon_size': None,
 'primary_color': '#2a4fd5',
 'active_user_count': 4,
 'icon_img': '',
 'display_name_prefixed': 'r/MentalH

In [22]:
# define function to fetch posts from subreddit ('MentalHealthSupport')
def fetch_posts(subreddit, category='hot', limit=None):
    if category == 'top':
        posts = subreddit.top(time_filter='year', limit=limit)
    elif category == 'rising':
        posts = subreddit.rising(limit=limit)
    elif category == 'new':
        posts = subreddit.new(limit=limit)
    else:
        posts = subreddit.hot(limit=limit)
    
    rows = []
    for submission in posts:
        row = {
            'title': submission.title,
            'score': submission.score,
            'upvote_ratio': submission.upvote_ratio,
            'created_utc': submission.created_utc,
            'selftext': submission.selftext,
            'subreddit': submission.subreddit,
            'author': str(submission.author),
            'media_only': submission.media_only,
            'permalink': submission.permalink,
            'num_comments': submission.num_comments
        }
        rows.append(row)
    
    return pd.DataFrame(rows)

In [23]:
mental_health_support_top = fetch_posts(subreddit, category='top')
mental_health_support_hot = fetch_posts(subreddit, category='hot')
mental_health_support_rising = fetch_posts(subreddit, category='rising')
mental_health_support_new = fetch_posts(subreddit, category='new')

In [24]:
# concatenate dataframes
mental_health_support_df = pd.concat([mental_health_support_top, 
                                      mental_health_support_hot, 
                                      mental_health_support_rising, 
                                      mental_health_support_new], 
                                     ignore_index=True)

# drop duplicates
mental_health_support_df = mental_health_support_df.drop_duplicates()

In [25]:
mental_health_support_df.head()

,title,score,upvote_ratio,created_utc,selftext,subreddit,author,media_only,permalink,num_comments
0,I feel hopeless.,157,0.99,1.709293e+09,"Hi, I’m 24F. I stay home 24/7 , everyday. I do...",MentalHealthSupport,Odd-View-667,False,/r/MentalHealthSupport/comments/1b3s6ad/i_feel...,154
1,What changes your negative thoughts/ has made ...,45,1.00,1.709444e+09,I am learning that affirmations and the like c...,MentalHealthSupport,Bananasloog1998,False,/r/MentalHealthSupport/comments/1b59lzz/what_c...,21
2,what is the point of living?,46,0.94,1.709354e+09,\ncontent warning for suicide.\n\n\n\nI dont u...,MentalHealthSupport,penjamincranklin,False,/r/MentalHealthSupport/comments/1b4fkko/what_i...,24
3,I need a hug,24,0.95,1.709557e+09,I wanna end me.,MentalHealthSupport,Positive-Parthlow,False,/r/MentalHealthSupport/comments/1b6a8vz/i_need...,26
4,What are you like after a hug,23,1.00,1.709738e+09,If I hug someone I just try my best not to cry...,MentalHealthSupport,itsRileyigitbanned,False,/r/MentalHealthSupport/comments/1b8299d/what_a...,17


In [26]:
# calculate upvotes and downvotes
mental_health_support_df['upvotes']=mental_health_support_df['score']/mental_health_support_df['upvote_ratio']
mental_health_support_df['downvotes']=mental_health_support_df['upvotes']-mental_health_support_df['score']
mental_health_support_df['upvotes/subscribers']=mental_health_support_df['upvotes']/subreddit.subscribers

mental_health_support_df.sample(5)

,title,score,upvote_ratio,created_utc,selftext,subreddit,author,media_only,permalink,num_comments,upvotes,downvotes,upvotes/subscribers
2017,I don’t trust anyone. Three girls (friends) ju...,3,0.81,1.717538e+09,Long story short: a guy hurt me emotionally. \...,MentalHealthSupport,PeaceQueen808,False,/r/MentalHealthSupport/comments/1d89api/i_dont...,1,3.703704,0.703704,0.000080
1748,I need help,1,1.00,1.715390e+09,Dont know where else too go. I'm dealing with ...,MentalHealthSupport,Standard_Ad1613,False,/r/MentalHealthSupport/comments/1cp5flu/i_need...,0,1.000000,0.000000,0.000022
1691,Extreme cptsd and not a supportive soul,1,1.00,1.715616e+09,"I am having EXTREME pain, vertigo, and nausea ...",MentalHealthSupport,NegativeInfluence_23,False,/r/MentalHealthSupport/comments/1cr2qwh/extrem...,4,1.000000,0.000000,0.000022
268,I can't tell anyone I don't have any motivation,5,1.00,1.709434e+09,I'm a teen in high-school and I'm currently st...,MentalHealthSupport,Rowdy-Rios,False,/r/MentalHealthSupport/comments/1b56a9e/i_cant...,4,5.000000,0.000000,0.000108
1195,I'm tired of living,1,1.00,1.717133e+09,I had a dream where I finally killed myself an...,MentalHealthSupport,Key-Ad4797,False,/r/MentalHealthSupport/comments/1d4o3z6/im_tir...,0,1.000000,0.000000,0.000022


In [27]:
# create csv 
mental_health_support_df.to_csv('data/mental_health_support_posts.csv', index=False)

In [28]:
mental_health_support_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1910 entries, 0 to 2801
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                1910 non-null   object 
 1   score                1910 non-null   int64  
 2   upvote_ratio         1910 non-null   float64
 3   created_utc          1910 non-null   float64
 4   selftext             1910 non-null   object 
 5   subreddit            1910 non-null   object 
 6   author               1910 non-null   object 
 7   media_only           1910 non-null   bool   
 8   permalink            1910 non-null   object 
 9   num_comments         1910 non-null   int64  
 10  upvotes              1910 non-null   float64
 11  downvotes            1910 non-null   float64
 12  upvotes/subscribers  1910 non-null   float64
dtypes: bool(1), float64(5), int64(2), object(5)
memory usage: 195.8+ KB


#### Combine Dataframes (r/MentalHealth and  r/MentalHealthSupport)

In [29]:
# combine both df into one
df = pd.concat([mental_health_support_df, 
                mental_health_df],
               ignore_index=True)

In [30]:
# create csv 
df.to_csv('data/reddit_posts.csv', index=False)

In [31]:
df.info()

df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3943 entries, 0 to 3942
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                3943 non-null   object 
 1   score                3943 non-null   int64  
 2   upvote_ratio         3943 non-null   float64
 3   created_utc          3943 non-null   float64
 4   selftext             3943 non-null   object 
 5   subreddit            3943 non-null   object 
 6   author               3943 non-null   object 
 7   media_only           3943 non-null   bool   
 8   permalink            3943 non-null   object 
 9   num_comments         3943 non-null   int64  
 10  upvotes              3943 non-null   float64
 11  downvotes            3943 non-null   float64
 12  upvotes/subscribers  3943 non-null   float64
dtypes: bool(1), float64(5), int64(2), object(5)
memory usage: 373.6+ KB


,title,score,upvote_ratio,created_utc,selftext,subreddit,author,media_only,permalink,num_comments,upvotes,downvotes,upvotes/subscribers
1139,I’m so depressed and down,1,1.00,1.717304e+09,For starters I’m going to preface to say I hav...,MentalHealthSupport,Diligent-Jackfruit-9,False,/r/MentalHealthSupport/comments/1d65mux/im_so_...,0,1.000000,0.000000,0.000022
3742,Help/advice needed for my partner,1,1.00,1.717934e+09,"Hello all, \n\nFirst post from a longtime lurk...",mentalhealth,Single-Jaguar-6509,False,/r/mentalhealth/comments/1dbrz3l/helpadvice_ne...,2,1.000000,0.000000,0.000002
851,Does anybody know what this is called/ what is...,1,0.67,1.710163e+09,Last night I had this like sudden episode wher...,MentalHealthSupport,CheeseMasterHeeHee,False,/r/MentalHealthSupport/comments/1bc2ulf/does_a...,1,1.492537,0.492537,0.000032
3305,Mild Depression (probably),1,1.00,1.718086e+09,TW// SMALL MENTION OF DISORDERED EATING\n\nHey...,mentalhealth,AnonymousCherryCoke,False,/r/mentalhealth/comments/1dd7662/mild_depressi...,0,1.000000,0.000000,0.000002
916,No one cares,1,1.00,1.718014e+09,No one cares\n\nI’m a teenager half way throug...,MentalHealthSupport,Cold-Entrance-4192,False,/r/MentalHealthSupport/comments/1dchxba/no_one...,0,1.000000,0.000000,0.000022


Tokenization/normalization

In [32]:
# Tokenization and Normalization
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


In [33]:
def preprocess_text(text):
    # Tokenize
    tokens = word_tokenize(text.lower())
    # Remove stop words and punctuation
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    # Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

In [36]:
# Apply preprocessing to the 'selftext' column
df['tokens'] = df['selftext'].apply(lambda x: preprocess_text(x) if isinstance(x, str) else [])

# Descriptive stats
df['token_count'] = df['tokens'].apply(len)
numerical_stats = df.describe()
token_count_stats = df['token_count'].describe()

# Most common words
all_tokens = [token for tokens in df['tokens'] for token in tokens]
fdist = FreqDist(all_tokens)
most_common_words = fdist.most_common(10)

# print stats
print("Numerical Statistics:\n", numerical_stats)
print("\nToken Count Statistics:\n", token_count_stats)
print("\nMost Common Words:\n", most_common_words)

Numerical Statistics:
              score  upvote_ratio   created_utc  num_comments      upvotes  \
count  3943.000000   3943.000000  3.943000e+03   3943.000000  3943.000000   
mean     29.661933      0.963338  1.711145e+09     16.971342    31.138814   
std      70.570435      0.084501  9.288383e+06     39.461661    73.623172   
min       0.000000      0.130000  1.680765e+09      0.000000     0.000000   
25%       1.000000      0.970000  1.709685e+09      0.000000     1.000000   
50%       2.000000      1.000000  1.715603e+09      1.000000     2.000000   
75%      30.000000      1.000000  1.717951e+09     13.000000    31.709719   
max     924.000000      1.000000  1.718242e+09    520.000000   942.857143   

         downvotes  upvotes/subscribers  token_count  
count  3943.000000          3943.000000  3943.000000  
mean      1.476881             0.000094    95.334263  
std       4.763071             0.000166    95.831406  
min       0.000000             0.000000     0.000000  
25%     